In [1]:
import sys
sys.path.append(sys.path[0]+'/../')

import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader

from dataloaders.minc_dataloader import MINCDataset, MINCDataLoader
from utils.loss_resnet import LossRN



In [2]:
minc_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc'
labels_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc/train.txt'
labels_path_t = '/home/ahmed/workspace/notebook/matrec/datasets/minc/test.txt'
BATCH_SIZE = 4
TRAIN_ITER = 10000
TEST_ITER = 400
start = 220000
lr = 0.0000005
size = 256

train_dataloader = MINCDataLoader(minc_path, labels_path, batch_size=BATCH_SIZE, size=size, f=0.16)
dataset_test = MINCDataset(minc_path, labels_path_t, size=(size, size))
test_loader = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)

In [3]:
from torchvision.models import googlenet, GoogLeNet_Weights

checkpoint = "../weights/googlenet_minc.pth"

model = googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 23, bias=True)
model.load_state_dict(torch.load(checkpoint), strict=False)

model = model.train()
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = LossRN()

In [ ]:
for epc in range(1,3):
    r = tqdm(range(start, len(train_dataloader)), leave=False, desc="Epoch {} starting from iteration {}: ".format(epc, start), 
                                                                     total=len(train_dataloader)-start)    
    
    for idx in r:
        x, y = train_dataloader[idx]
        y_pred = model(x.cuda())
        lf = loss.compute(y_pred, y)
        lf.backward()
        optimizer.step()
        optimizer.zero_grad()
        r.set_postfix(loss=lf.item())
        
        if idx % TRAIN_ITER == 0 and idx != start:
            #save checkpoint
            torch.save(model.state_dict(), checkpoint)
            #test loss run on val
            with torch.no_grad():
                ac = 0
                for i in range(TEST_ITER):
                    x, y = next(iter(test_loader))
                    y_pred = model(x.cuda())
                    ac = ac + loss.accuracy(y_pred, y)
                ac = ac/TEST_ITER
                print('Test N ' + str(epc) + '-' + str(idx//TRAIN_ITER) + ' : ' + str(float(ac)) + '\n')
        start = 0

Epoch 1 starting from iteration 300000:  18%|█▊        | 10003/56502 [11:51<60:03:10,  4.65s/it, loss=0.0495]

Test N 1-31 : 72.25



Epoch 1 starting from iteration 300000:  35%|███▌      | 20002/56502 [24:04<66:13:42,  6.53s/it, loss=0.114] 

Test N 1-32 : 72.9375



Epoch 1 starting from iteration 300000:  53%|█████▎    | 30003/56502 [36:28<30:44:17,  4.18s/it, loss=1.45]  

...

Epoch 2 starting from iteration 0:  59%|█████▉    | 210003/356502 [4:23:39<207:05:14,  5.09s/it, loss=0.449] 

Test N 2-21 : 74.375



Epoch 2 starting from iteration 0:  62%|██████▏   | 220004/356502 [4:35:38<162:57:30,  4.30s/it, loss=0.357] 

Test N 2-22 : 73.75



Epoch 2 starting from iteration 0:  62%|██████▏   | 221983/356502 [4:37:54<2:10:25, 17.19it/s, loss=0.142]  